In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.model_selection as skm
import statsmodels.formula.api as sm
import scipy.stats as stats
import folium as fo
from folium import plugins
from folium.plugins import HeatMap

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
"""
    O objetivo dessa analise é prever o preço das casas atraves de uma regressão linear multipla afim de aplicar os conhecimentos que
    adiquiri de python e estatistica.
    
    Colunas:

    Id: a notation for a house

    Date: Date house was sold

    Price: Price is prediction target

    Bedrooms: Number of Bedrooms/House

    Bathrooms: Number of bathrooms/House

    Sqft_Living: square footage of the home

    Sqft_Lot: square footage of the lot

    Floors: Total floors (levels) in house

    Waterfront: House which has a view to a waterfront

    View: Has been viewed

    Condition: How good the condition is ( Overall )

    Grade: overall grade given to the housing unit, based on King County grading system

    Sqft_Above: square footage of house apart from basement

    Sqft_Basement: square footage of the basement

    Yr_Built: Built Year

    Yr_Renovated: Year when house was renovated

    Zipcode: Zip

    Lat: Latitude coordinate

    Long: Longitude coordinate

    Sqft_Living15: Living room area in 2015(implies-- some renovations) This might or might not have affected the lotsize area

    Sqft_Lot15: lotSize area in 2015(implies-- some renovations)
"""

In [ ]:
base = pd.read_csv(r'/kaggle/input/housesalesprediction/kc_house_data.csv')

#remover coluna de id
del base['id']

In [ ]:
# resumo das colunas
base.describe()

Algumas informações da tabela acima me chamaram atenção, existem casas com zero quartos e zero banheiros, também existe uma casa com 33 banheiros, vou filtrar essas informações para analisa-las

In [ ]:
base.loc[(base['bathrooms'] == 0) | (base['bedrooms'] == 0) | (base['bedrooms'] == 33), :]

1. - A casa que tem 33 quartos tem um sqft_living não tão grande, a coluna sqft_living contém a metragem quadrada da casa, vou considerar 
    que houve um erro de digitação e o valor correto é 3.
    
   - Quanto aos registros que não tem quarto ou banheiro irei remove-los da analise.

In [ ]:
index = base.loc[(base['bathrooms'] == 0) | (base['bedrooms'] == 0), :].index
base = base.drop(index=index)

base.loc[base['bedrooms'] == 33, 'bedrooms'] = 3

In [ ]:
"""
    Abaixo vou criar algumas variaveis que eu acredito que podem ter impacto no preço da casa
"""

"""Eu tenho duvida no significado das variavels sqft_lot15 e sqft_living15, de acordo com o link abaixo eu entendo que essas variaveis
representariam a media da metragem dos 15 imoveis mais proximos, vou considerar que essas variaveis tem o significado do link abaixo.
https://www.slideshare.net/PawanShivhare1/predicting-king-county-house-prices
"""
    
base['maior_liv15'] = 0 # variavel dummy para verificar se a casa tem metragem maior que os imoveis mais proximos
base['maior_lot15'] = 0 # variavel dummy para verificar se a metragem total do terreno é maior que as 15 casas mais proximas
base['reformado'] = 0 # variavel dummy que indica se o imovel foi reformado
base['ano_venda'] = None # ano que o imovel foi vendido
base['idade'] = None # idade do imovel 
base['ano_considerado'] = None # se o imovel foi reformado então o ano considerado na analise sera o ano da reforma e não o ano da construção

base['date'] = pd.to_datetime(base['date'])
base['ano_venda'] = base['date'].dt.year
base['ano_considerado'] = base[['yr_built', 'yr_renovated']].apply(lambda x: x['yr_renovated'] if x['yr_renovated'] > 0 else x['yr_built'], axis=1)
base['idade'] = base['ano_venda'] - base['ano_considerado']
base['reformado'] = base[['yr_built', 'yr_renovated']].apply(lambda x: 1 if x['yr_renovated'] > 0 else 0, axis=1)

base.loc[base.sqft_living >= base.sqft_living15, 'maior_liv15'] = 1
base.loc[base.sqft_lot >= base.sqft_lot15, 'maior_lot15'] = 1

del base['date']

In [ ]:
mapa = fo.Map(location=[np.mean(base['lat']), np.mean(base['long'])])
base_agg = base[['lat', 'long', 'price']].groupby(['lat', 'long'], as_index=False).mean().values.tolist()
HeatMap(base_agg, radius=8, max_zoom =13, blur=15).add_to(mapa);
mapa

In [ ]:
# criação das bases de treino e teste
treino, teste = skm.train_test_split(base, test_size=0.3, train_size=0.7)

In [ ]:
treino.head(5)

In [ ]:
"""
    Matrix de correção para analisar quais as colunas tem maior correlçao com o preço dos imoveis e a correlçao existente entre as
    colunas
"""

correlacao = treino.corr()
fig = plt.figure(figsize=(20,10))
#cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(correlacao, annot=True);

As variaveis que possuem maior correção com o preço são: bedrooms, bathrooms, sqft_living, floors, waterfront, view, grade, sqft_above,
sqft_basement, lat, sqft_living15, maior_liv15

abaixo criarei outra matrix de correção apenas com as variaveis filtradas acima

In [ ]:
colunas = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront', 'view', 'grade', 'sqft_above', 
           'sqft_basement', 'lat', 'sqft_living15', 'maior_liv15']

correlacao = treino[colunas].corr()
fig = plt.figure(figsize=(20,5))
#cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(correlacao, annot=True);

Aparentemente existe multicolinearidade nessa matrix, a multicolinearidade ocorre quando existe forte correlção entre variaveis independentes.

http://www.portalaction.com.br/analise-de-regressao/36-analise-de-colinearidade-e-multicolinearidade

A variavel sqft_living esta bastante correlacionada com a variavel bathrooms, sqft_above e sqft_living15, grade.
inicialmente vou desconsiderar a varivael sqft_above pois ela apresenta a correlação mais forte com sqft_living.

In [ ]:
colunas = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'floors', 'waterfront', 'view', 'grade', 
           'sqft_basement', 'lat', 'sqft_living15', 'maior_liv15']

In [ ]:
# analise da relação entre as variaveis
sns.pairplot(treino[colunas]);

In [ ]:
# a regressão abaixo será feita com as variaveis que constam na lista chamada "colunas" sem nenhum ajuste

colunas_temp = colunas.copy()
colunas_temp.remove('price')
formula = 'price ~ ' + ' + '.join(colunas_temp)
modelo = sm.ols(formula=formula, data=treino).fit()
print(modelo.summary())

In [ ]:
residuos = modelo.resid
previsto = modelo.predict()

sns.scatterplot(x=previsto, y=residuos)

In [ ]:
residuos.hist();

In [ ]:
plot = plt.subplot()
stats.probplot(residuos, dist="norm", plot=plot);

Algumas observações sobre os resultados acima: 

-as variaveis bathrooms e sqft_basement são significativas.

-a analise dos residuos revela presença de heterocedasticidade

-os residuos não estão normalmente distribuidos

De acordo com a pagina https://blog.minitab.com/blog/adventures-in-statistics-2/how-important-are-normal-residuals-in-regression-analysis
as previsões geradas por um modelo de regressão linear que não possui residuos normalmente distribuidos pode apresentar resultados imprecisos.

Para tentar melhorar a linearidade do modelo vou aplicar ln em algumas variaveis.

In [ ]:
formula = 'np.log(price) ~ waterfront + view + maior_liv15 + grade + lat + \
           np.log(1+sqft_above) + np.log(1+sqft_living) + np.log(1+sqft_basement) + bedrooms'

modelo = sm.ols(formula=formula, data=treino).fit()
print(modelo.summary())

In [ ]:
residuos = modelo.resid
previsto = modelo.predict()

sns.scatterplot(x=previsto, y=residuos)

In [ ]:
residuos.hist();

In [ ]:
plot = plt.subplot()
stats.probplot(residuos, dist="norm", plot=plot);

Ao aplicar o ln nas variaveis, como em np.log(1+sqft_basement), eu somei 1 pois algumas variaveis apresentam valores zerados como por exemplo casas que não tem porão, ao somar 1 não ocorre erro no ln e a proporção entre os valores se mantem, pois se ln(a) > ln(b) então ln(1+a) > ln(1+b).

Aparentemente os residuos estão normalmente distribuidos e o problema da heterocedasticidade foi corrigido.

O objetivo agora é usar esse modelo para prever os preços da base de teste

In [ ]:
teste['previsao'] = None # coluna que armazena os valores previstos pelo modelo
teste['dif'] = None # o resultado dessa coluna sera a diferença percentual entre o previsto e o realizado
teste['acc'] = 'n' # para validar a qualidade do modelo vou contar quantos registros tem margem de erro de no maximo 10% pra mais ou pra menos

teste.loc[:, 'previsao'] = modelo.predict(exog=teste)
teste.loc[:, 'previsao'] = np.exp(teste['previsao'])
teste.loc[:, 'dif'] = ((teste['previsao']/teste['price']) - 1) * 100
teste.loc[(teste.dif <= 10) & (teste.dif >= -10), 'acc'] = 's'
teste[['acc']].groupby(['acc']).size()


In [ ]:
previsto = teste['previsao']
resid = teste['price'] - teste['previsao']
sns.scatterplot(x=previsto, y=resid)